# WASP Module Test



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: %s" % device)

device: cpu


In [3]:
from wasp import WASP
in_t = torch.randn(16, 1280, 120, 90)

w = WASP(device)
w

WASP(
  (convolutions): ModuleList(
    (0): ModuleDict(
      (3x3): Conv2d(1280, 256, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6))
      (1x1): Sequential(
        (0): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(256, 60, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (1): ModuleDict(
      (3x3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(12, 12), dilation=(12, 12))
      (1x1): Sequential(
        (0): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(256, 60, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (2): ModuleDict(
      (3x3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(18, 18), dilation=(18, 18))
      (1x1): Sequential(
        (0): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(256, 60, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (3): ModuleDict(
      (3x3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=

In [4]:
out = w(in_t)
print(out.shape)

torch.Size([16, 256, 120, 90])
